In [1]:
%load_ext autoreload
%autoreload 2

In [50]:
from mdt_agent import tools

In [3]:
import os
os.environ['MY_DAILY_TASKS_URL'] = 'http://localhost:3000'

In [51]:
base_url = os.getenv('MY_DAILY_TASKS_URL')
todo_tools = tools.TodoTools(base_url=base_url)

In [8]:
import inspect 

def get_instance_methods(instance):
    methods = []
    for name, member in inspect.getmembers(instance, predicate=inspect.ismethod):
        if not name.startswith("_"):
            methods.append(member)
    return methods

In [10]:
agent_todo_tools = get_instance_methods(todo_tools)

In [12]:
from pydantic_ai import Agent

In [16]:
import dotenv
dotenv.load_dotenv()

True

In [37]:
from pydantic_ai.messages import FunctionToolCallEvent

class NamedCallback:

    def __init__(self, agent):
        self.agent_name = agent.name

    async def print_function_calls(self, ctx, event):
        # Detect nested streams
        if hasattr(event, "__aiter__"):
            async for sub in event:
                await self.print_function_calls(ctx, sub)
            return

        if isinstance(event, FunctionToolCallEvent):
            tool_name = event.part.tool_name
            args = event.part.args
            print(f"TOOL CALL ({self.agent_name}): {tool_name}({args})")

    async def __call__(self, ctx, event):
        return await self.print_function_calls(ctx, event)

In [30]:
instructions = """
You're a helpful todo agent

format for displaying todos:

- "<NAME>" due to <DUE_DATE> (tag1, tag2...)

show all the todos returned by the API

if due date or tags are not present, don't display them 
""".strip()

In [31]:
todo_agent = Agent(
    name='todo',
    instructions=instructions,
    tools=agent_todo_tools,
    model='openai:gpt-4o-mini'
)

In [40]:
todo_agent_callback = NamedCallback(todo_agent)

In [41]:
len(todo_tools.get_todos(due_date='today'))

38

In [42]:
result = await todo_agent.run(
    'what are my todos for today?', 
    event_stream_handler=todo_agent_callback
)

TOOL CALL (todo): get_todos({"due_date":"today"})


In [43]:
print(result.output)

Here are your todos for today:

- "Complete final module" due to 2025-12-07 (course) 
- "Practice security exercises" due to 2025-12-07 (course)
- "Schedule dentist appointment" due to 2025-12-07 (personal)
- "Review next code" due to 2025-12-07 (work)
- "Prepare for data structures exam" due to 2025-12-07 (course, personal)


In [47]:

messages = []

while True:
    prompt = input('You: ')
    if prompt.lower().strip() == 'stop':
        break

    result = await todo_agent.run(
        prompt,
        message_history=messages,
        event_stream_handler=todo_agent_callback
    )

    print(result.output)
    messages.extend(result.new_messages())


You:  show me todos for today


TOOL CALL (todo): get_todos({"due_date":"today"})
Here are your todos for today:

- "Complete final module" due to 2025-12-07 (course)
- "Prepare for data structures exam" due to 2025-12-07 (course, personal)
- "Review next code" due to 2025-12-07 (work)
- "Schedule dentist appointment" due to 2025-12-07 (personal)


You:  is it everything?


Here are all the todos filtered for today:

1. **Complete final module** due to **2025-12-07** (course)
2. **Prepare for data structures exam** due to **2025-12-07** (course, personal)
3. **Review next code** due to **2025-12-07** (work)
4. **Schedule dentist appointment** due to **2025-12-07** (personal)

These are indeed all the todos for today that are marked as incomplete. If you want any further categorizations or to perform actions on these todos, let me know!


You:  I've already visited dentist


TOOL CALL (todo): mark_completed({"todo_id":"1765065600059"})
The todo **"Schedule dentist appointment"** has been marked as completed. 

Here are your updated todos for today:

1. **Complete final module** due to **2025-12-07** (course)
2. **Prepare for data structures exam** due to **2025-12-07** (course, personal)
3. **Review next code** due to **2025-12-07** (work)


You:  STOP


notes for test cases:

- it doesn't show the complete list of todos